In [61]:
#Header to be adjusted according to the background used

HEADER_HEIGHT = 250  # Header space in pixels
BACKGROUND_IMAGE_PATH = "saintvalentin.png"  # Path to your A4 design

In [62]:
import os
import requests
from PIL import Image, ImageDraw, ImageFont, ImageOps
from oauth2client.service_account import ServiceAccountCredentials
import gspread

# Constants
GRID_ROWS = 3
GRID_COLS = 3
PAGE_WIDTH, PAGE_HEIGHT = 1414, 2000  # A4 size at 300 DPI
CELL_PADDING = 100
GRID_WIDTH = PAGE_WIDTH // GRID_COLS
GRID_HEIGHT = (PAGE_HEIGHT - HEADER_HEIGHT) // GRID_ROWS


# Google Sheets Setup
SCOPES = ['https://www.googleapis.com/auth/drive', 'https://www.googleapis.com/auth/spreadsheets']
GOOGLE_SHEET_NAME = "Catalogue client"
GOOGLE_CREDENTIALS_FILE = "leo-omega-5ffcec1345ff.json"

In [63]:
def fetch_google_sheet_data(sheet_name):
    credentials = ServiceAccountCredentials.from_json_keyfile_name(GOOGLE_CREDENTIALS_FILE, SCOPES)
    client = gspread.authorize(credentials)
    sheet = client.open(sheet_name).sheet1
    return sheet.get_all_records()

In [64]:
def convert_to_direct_link(drive_link):
    if "drive.google.com" in drive_link:
        file_id = drive_link.split("/d/")[1].split("/")[0]
        return f"https://drive.google.com/uc?export=download&id={file_id}"
    return drive_link

In [65]:
def download_and_resize_image(image_url, width, height):
    direct_link = convert_to_direct_link(image_url)
    response = requests.get(direct_link, stream=True)
    response.raise_for_status()
    image = Image.open(response.raw)
    image = ImageOps.contain(image, (int(width), int(height)), Image.Resampling.LANCZOS)
    return image

In [ ]:
def format_price(price):
    """Ajoute des espaces dans le prix pour lisibilité (ex: 1234 -> '1 234 €')."""
    return f"{price:,.0f}".replace(",", " ")

In [66]:
def generate_catalog(data, output_folder, header_text="Product Catalog"):
    font_name = ImageFont.truetype("Skranji/Skranji-Bold.ttf", 28)
    font_price = ImageFont.truetype("arial-font/ARIBL0.ttf", 36)
    #font_header = ImageFont.truetype("arialbd.ttf", 60)

    page_number = 1
    items_per_page = GRID_ROWS * GRID_COLS
    os.makedirs(output_folder, exist_ok=True)

    for page_start in range(0, len(data), items_per_page):
        # Load the background image
        background = Image.open(BACKGROUND_IMAGE_PATH).convert("RGBA")
        draw = ImageDraw.Draw(background)

        # Draw grid content
        for index, item in enumerate(data[page_start:page_start + items_per_page]):
            row = index // GRID_COLS
            col = index % GRID_COLS

            # Top-left corner of the cell
            cell_x = col * GRID_WIDTH
            cell_y = HEADER_HEIGHT + row * GRID_HEIGHT

            # Calculate center of the cell
            cell_center_x = cell_x + GRID_WIDTH // 2
            cell_center_y = cell_y + GRID_HEIGHT // 2

            try:
                # Resize and position the image
                image = download_and_resize_image(item['Photo'], GRID_WIDTH - 2 * CELL_PADDING, GRID_HEIGHT - 150)
                img_width, img_height = image.size
                img_x = cell_center_x - img_width // 2
                img_y = cell_center_y - img_height // 2 - 50  # Adjust upward for space for the price below
                background.paste(image, (img_x, img_y), mask=image.convert("RGBA"))

                # Measure text sizes
                name_text = item['Name'][:20]
                price_text = f"{format_price(item['Price'])} Ar"
                
                name_bbox = font_name.getbbox(name_text)
                name_width, name_height = name_bbox[2], name_bbox[3]

                price_bbox = font_price.getbbox(price_text)
                price_width, price_height = price_bbox[2], price_bbox[3]

                # Draw product name (centered above the image)
                name_x = cell_center_x - name_width // 2
                name_y = img_y - name_height - 20  # Slight gap above the image
                draw.text((name_x, name_y), name_text, font=font_name, fill="black")

                # Draw product price (centered below the image)
                price_x = cell_center_x - price_width // 2
                price_y = img_y + img_height + 10  # Slight gap below the image
                draw.text((price_x, price_y), price_text, font=font_price, fill="red")

            except Exception as e:
                print(f"Error downloading or placing image for {item['Name']}: {e}")

        # Save the page as an image
        output_path = os.path.join(output_folder, f"page{page_number}.png")
        background.save(output_path)
        print(f"Page {page_number} saved as {output_path}")
        page_number += 1

In [67]:
if __name__ == "__main__":
    sheet_data = fetch_google_sheet_data(GOOGLE_SHEET_NAME)
    output_folder = "catalog_pages"
    generate_catalog(sheet_data, output_folder)

#sheet_data: array of dict [{'Name': 'Congelateur 110L', 'Price': '2.000.000 Ar', 'Photo': '...'}, {……}]

Page 1 saved as catalog_pages/page1.png
Page 2 saved as catalog_pages/page2.png
